# Assignment 3

**Joris LIMONIER**

_Note:_ For scrolling-time reasons, this notebook only contains calls to functions. The actual core of function is in the `assignment_utils.py` file.


In [35]:
import pandas as pd
import numpy as np
import stan
from assignment_utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Exercise

In this exercise, you will use the ADNI dataset from the past lesson.

---

### Preparation

We first load the data and describe it.


In [2]:
adni = ADNI()
adni.diag.describe()


,RID,APOE4,DX,AGE,WholeBrain.bl,ICV,norm_brain
count,826.000000,826.000000,826.000000,826.000000,8.260000e+02,8.260000e+02,826.000000
mean,2686.322034,0.525424,0.388620,74.451574,1.011453e+06,1.521185e+06,-0.001905
std,2062.148046,0.658710,0.487732,6.648689,1.113623e+05,1.680556e+05,1.000892
min,2.000000,0.000000,0.000000,55.100000,7.274780e+05,1.100687e+06,-2.765395
25%,673.250000,0.000000,0.000000,70.500000,9.329465e+05,1.396231e+06,-0.719043
50%,2718.000000,0.000000,0.000000,74.150000,1.008351e+06,1.504898e+06,0.022915
75%,4690.500000,1.000000,1.000000,78.900000,1.087573e+06,1.634110e+06,0.684545
max,5296.000000,2.000000,1.000000,90.900000,1.486036e+06,2.057399e+06,3.236658


We note that the `AGE` variable is distributed very similar to a Gaussian. For this reason, we will

In [3]:
adni.plot_kde_vs_norm()

In [4]:
adni.plot_apoe4()

---


### Question 1

Fit a model to predict the diagnosis (DX) of the subjects using both AGE and APOE4 as predictors.
#### Answer


In [5]:
# Prepare C code that will be passed to Stan
code_to_stan = """
data {
  int<lower=1> N;
  int y[N];
  real x1[N];
  real x2[N];
}
parameters {
  real a;
  real b;
  real c;
}
transformed parameters {
  vector[N] p_i;
  for (i in 1:N) {
    p_i[i] = exp(a + b * x1[i] + c * x2[i])/(1 + exp(a + b * x1[i] + c * x2[i])); 
    }
}
model {
  a ~ normal(0, 3);
  b ~ normal(0, 3);
  c ~ normal(0, 3);
  y ~ binomial(1, p_i);
}
"""

posterior = adni.run_stan_model(features=["AGE", "APOE4"], program_code=code_to_stan, num_samples=1000)
posterior

Building...

/home/joris/.cache/httpstan/4.9.0/models/vk4ds7lo/model_vk4ds7lo.cpp: In constructor ‘model_vk4ds7lo_namespace::model_vk4ds7lo::model_vk4ds7lo(stan::io::var_context&, unsigned int, std::ostream*)’:
/home/joris/.cache/httpstan/4.9.0/models/vk4ds7lo/model_vk4ds7lo.cpp:67:11: warning: variable ‘pos__’ set but not used [-Wunused-but-set-variable]
   67 |       int pos__ = std::numeric_limits<int>::min();
      |           ^~~~~
In file included from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/prim/fun.hpp:124,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun/multiply.hpp:7,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun/elt_multiply.hpp:9,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun.hpp:55,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/r


Building: 24.8s, done.Messages from stanc:
Warning in '/tmp/httpstan_a7o4q4c3/model_vk4ds7lo.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_a7o4q4c3/model_vk4ds7lo.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_a7o4q4c3/model_vk4ds7lo.stan', line 6, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to


<class 'stan.fit.Fit'>


<stan.Fit>
Parameters:
    a: ()
    b: ()
    c: ()
    p_i: (826,)
Draws: 4000

In [6]:
# Save results in a dict
waic_res = {"AGE + APOE4": adni.get_waic(fit=posterior)}


  0%|          | 0/826 [00:00<?, ?it/s]

In [7]:
waic_res

{'AGE + APOE4': 951.8712633945672}

In [8]:
adni.get_params_box_plot(fit=posterior, model_params=["a", "b", "c"])

We notice from the box plot above that the `b` parameter seems to play a very minor role in the computation of the `DX` variable, whereas `a` and `c` seem to be non-negligeable, with a lot of variability for `a`.

---


### Question 2

Consider subjects who are 80 years old and check the effect of the APOE4 gene on the diagnosis.

Hint: You'll draw many samples from two binomial distributions. One where APOE4 is included in the computation of $p_i$ and one where it's not.


#### Answer

We first prepare the dataset with subjects that are exactly 80 only. \
We note that there are only 4 patients with this age, which is very little. Furthermore, only one of those 4 patients has developed Alzheimer's disease. This is close to what some researchers have to face in the medical field: a large number of features for very few observations, with only some of those patients that are sick.


In [9]:
adni.eighty

,RID,APOE4,DX,AGE,WholeBrain.bl,ICV,norm_brain
139,230,0.0,0,80.0,1051053.0,1.714028e+06,-1.030522
489,866,0.0,0,80.0,943825.0,1.388961e+06,0.238778
522,920,1.0,0,80.0,946606.0,1.464818e+06,-0.398452
740,1285,1.0,1,80.0,1025968.0,1.626103e+06,-0.691147


We want to compare the two following models:

1. `DX ~ `$\varnothing$
1. `DX ~ APOE4`

The first model is a model containing only the parameter $a$ responsible for the intercept, that is:

$$
p_i = \frac{\exp(a)}{1 + \exp(a)}
$$

while the second model also contains a parameter $b$ responsible for an increase in the `APOE4` variable, that is:

$$
p_i = \frac{\exp(a + b x_i)}{1 + \exp(a + b x_i)}
$$


In [10]:
code_80_yo_intercept = """
data {
  int<lower=1> N;
  int y[N];
}
parameters {
  real a;
}
transformed parameters {
  vector[N] p_i;
  for (i in 1:N) {
    p_i[i] = exp(a)/(1 + exp(a)); 
    }
}
model {
  a ~ normal(0, 3);
  y ~ binomial(1, p_i);
}
"""
posterior_80_yo_intercept = adni.run_stan_model(
  features=[], program_code=code_80_yo_intercept, num_samples=5000, data_name="80 yo"
)
posterior_80_yo_intercept


Building...

/home/joris/.cache/httpstan/4.9.0/models/zicuhcqz/model_zicuhcqz.cpp: In constructor ‘model_zicuhcqz_namespace::model_zicuhcqz::model_zicuhcqz(stan::io::var_context&, unsigned int, std::ostream*)’:
/home/joris/.cache/httpstan/4.9.0/models/zicuhcqz/model_zicuhcqz.cpp:57:11: warning: variable ‘pos__’ set but not used [-Wunused-but-set-variable]
   57 |       int pos__ = std::numeric_limits<int>::min();
      |           ^~~~~
In file included from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/prim/fun.hpp:124,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun/multiply.hpp:7,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun/elt_multiply.hpp:9,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun.hpp:55,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/r


Building: 24.2s, done.Messages from stanc:
Warning in '/tmp/httpstan_vcudh77p/model_zicuhcqz.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Sampling:   0%
Sampling:  25% (6000/24000)
Sampling:  50% (12000/24000)
Sampling:  75% (18000/24000)
Sampling: 100% (24000/24000)
Sampling: 100% (24000/24000), done.
Messages received during sampling:
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would 

<class 'stan.fit.Fit'>


<stan.Fit>
Parameters:
    a: ()
    p_i: (4,)
Draws: 20000

In [11]:
waic_80_yo_intercept = adni.get_waic(
  fit=posterior_80_yo_intercept, data_name="80 yo", sample_size_waic=5000
)
waic_80_yo = {"intercept only": waic_80_yo_intercept}


  0%|          | 0/4 [00:00<?, ?it/s]

In [55]:
from textwrap import dedent

s = """
  data {
    int<lower=1> N;
    int y[N];
  }"""
pystan_code = adni._generate_pystan_code(
  param_distr={
    "a": "normal(0, 3)",
    # "b": "uniform(1, 3)",
  },
  p_i_formula="p_i[i] = exp(a)/(1 + exp(a));",
)
posterior_80_yo_intercept = adni.run_stan_model(
  features=[], program_code=pystan_code, num_samples=10, data_name="80 yo"
)
posterior_80_yo_intercept


Building...

/home/joris/.cache/httpstan/4.9.0/models/hvxuwstq/model_hvxuwstq.cpp: In constructor ‘model_hvxuwstq_namespace::model_hvxuwstq::model_hvxuwstq(stan::io::var_context&, unsigned int, std::ostream*)’:
/home/joris/.cache/httpstan/4.9.0/models/hvxuwstq/model_hvxuwstq.cpp:59:11: warning: variable ‘pos__’ set but not used [-Wunused-but-set-variable]
   59 |       int pos__ = std::numeric_limits<int>::min();
      |           ^~~~~
In file included from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/prim/fun.hpp:124,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun/multiply.hpp:7,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun/elt_multiply.hpp:9,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun.hpp:55,
                 from /home/joris/.local/lib/python3.10/site-packages/httpstan/include/stan/math/r


Building: 25.5s, done.Messages from stanc:
Warning in '/tmp/httpstan_wfprc2jj/model_hvxuwstq.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_wfprc2jj/model_hvxuwstq.stan', line 21, column 0: Parameter
    b is given a uniform distribution. The uniform distribution is not
    recommended, for two reasons: (a) Except when there are logical or
    physical constraints, it is very unusual for you to be sure that a
    parameter will fall inside a specified range, and (b) The infinite
    gradient induced by a uniform density can cause difficulties for Stan's
    sampling algorithm. As a consequence, we recommend soft constraints
    rather than hard constraints; for example, instead of giving an
    elasticity parameter a uniform(0,1) distribu

<class 'stan.fit.Fit'>


<stan.Fit>
Parameters:
    a: ()
    b: ()
    p_i: (4,)
Draws: 40

---



### Question 3

In the last lesson, we fitted a model to predict the diagnosis using only the size of the brain (norm_brain). Compare this model and the one of question 1 in terms of WAIC. Is one better than the other ?

#### Answer
